In [1]:
from aicsimageio import AICSImage

In [2]:
img = AICSImage("/Users/danielt/Downloads/AICS-12_143.ome.tif")

In [3]:
data = img.get_image_data()
print(data.shape)
print(img.channel_names)
print(img.physical_pixel_sizes)

(1, 9, 60, 624, 924)
['CMDRP', 'EGFP', 'H3342', 'Bright_100X', 'SEG_STRUCT', 'SEG_Memb', 'SEG_DNA', 'CON_Memb', 'CON_DNA']
PhysicalPixelSizes(Z=0.29, Y=0.10833333333333332, X=0.10833333333333332)


In [4]:
# prepare the metadata for ome-zarr

ch = []
for i in range(data.shape[1]):
    ch.append({"active": True,
        "coefficient": 1,
        "color": "0000FF",
        "family": "linear",
        "inverted": False,
        "label": img.channel_names[i],
        "window": {
            "end": float(data[0,i, :,:,:].max()),
            "max": float(data[0,i, :,:,:].max()),
            "min": float(data[0,i, :,:,:].min()),
            "start": float(data[0,i, :,:,:].min())
        }
    })

omero = {
    "id": 1,                              # ID in OMERO
    "name": "image0",                # Name as shown in the UI
    "version": "0.1",                     # Current version
    "channels": ch,
    "rdefs": {
        "defaultT": 0,                    # First timepoint to show the user
        "defaultZ": data.shape[2]//2,                  # First Z section to show the user
        "model": "color",                  # "color" or "greyscale"
        "projection": "normal"  # ?
    },
    "meta": {
        "projectDescription": "20+ lines of gene edited cells etc",
        "datasetName": "aics_hipsc_v2020.1",
        "projectId": 2,
        "imageDescription": "foo bar",
        "imageTimestamp": 1277977808.0,
        "imageId": 12,
        "imageAuthor": "danielt",
        "imageName": "AICS-12_143.ome.tif",
        "datasetDescription": "variance dataset after QC",
        "projectName": "aics cell variance project",
        "datasetId": 3
    },
    "pixel_size": {
        "y": img.physical_pixel_sizes.Y,
        "x": img.physical_pixel_sizes.X,
        "z": img.physical_pixel_sizes.Z
    },
    "size": {
        "width": data.shape[4],
        "c": data.shape[1],
        "z": data.shape[2],
        "t": data.shape[0],
        "height": data.shape[3]
    },
}


In [16]:
# do the writing

import shutil
import zarr
import pathlib
import numpy
import os
from ome_zarr.writer import write_image
from ome_zarr.scale import Scaler
from ome_zarr.io import parse_url

name = "z0.zarr"
mypath = pathlib.Path("./" + name)
shutil.rmtree(mypath)
print(mypath)
store = parse_url(mypath, mode="w").store
print(store)
root = zarr.group(store=store)
print(root)
group = root.create_group("image0")

scaler = Scaler()
write_image(data, group, chunks=(60,256,256), scaler=scaler, omero=omero)
print(os.listdir(mypath))
print(os.listdir(mypath / "image0"))

z0.zarr
<zarr.hierarchy.Group '/'>
['image0', '.zgroup']
['.zattrs', '.zgroup', '0', '1', '4', '3', '2']


In [6]:
from ome_zarr.reader import Multiscales, Reader
reader = Reader(parse_url(mypath / "image0"))
node = list(reader())[0]
print(node.data[0].shape)
# assert node.data[0].chunks == ((1,), (2,), (1,), (128, 128), (128, 128))

(1, 9, 60, 624, 924)


In [7]:
# levels
print(len(node.data))

5


In [8]:
print(node.data[1].shape)
print(node.data[2].shape)
print(node.data[3].shape)
print(node.data[4].shape)
print(node.data[4].dtype)
print(node.data[4][0].shape)


(1, 9, 60, 462, 312)
(1, 9, 60, 156, 231)
(1, 9, 60, 115, 78)
(1, 9, 60, 39, 57)
uint16
(9, 60, 39, 57)


In [15]:
import nbvv
level = 0
levelxyscale = 2**(level)
nbvv.volshow(node.data[level][0].compute(), spacing=(img.physical_pixel_sizes.X*levelxyscale,
        img.physical_pixel_sizes.Y*levelxyscale,
        img.physical_pixel_sizes.Z))

VolumeWidget(dimensions={'tile_width': 341, 'tile_height': 204, 'rows': 10, 'cols': 6, 'atlas_width': 2046, 'a…